In [2]:
#imports
import pandas as pd
import os
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import taxopy
from ete3 import NCBITaxa
%matplotlib inline

### Prepare NCBI all Viral Nucleotide: 

prepare the 'lineage' file required by sourmash

In [10]:
ncbi_viral_metadataPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/tools/databases/AllNuclMetadata.csv.gz"

In [11]:
ncbi_viral = pd.read_csv(ncbi_viral_metadataPath, usecols=[0,4,5,6,16,17]) #this file large select only useful cols

/tmp/ipykernel_3267704/1812190525.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  ncbi_viral = pd.read_csv(ncbi_viral_metadataPath, usecols=[0,4,5,6,16,17]) #this file large select only useful cols


In [12]:
ncbi_viral.shape


(12516324, 6)

In [13]:
ncbi_viral.head()

,#Accession,Species,Genus,Family,Host,Isolation_Source
0,NC_079174.1,Erwinia phage Fifi44,Loessnervirus,Chaseviridae,Erwinia amylovora,NaN
1,NC_079175.1,Escherichia phage PO103-1,Carltongylesvirus,Chaseviridae,Escherichia coli,NaN
2,NC_079176.1,Escherichia phage MLP1,Carltongylesvirus,Chaseviridae,Escherichia coli CFT073,NaN
3,NC_079177.1,Yersinia phage PYps3T,Carltongylesvirus,Chaseviridae,Yersinia pseudotuberculosis,NaN
4,NC_079178.1,Yersinia phage PYps23T,Carltongylesvirus,Chaseviridae,Yersinia pseudotuberculosis,NaN


In [14]:
#rename the Accession to ident as required by sourmash
ncbi_viral.rename(columns={"#Accession":"ident"}, inplace=True)

#add empty columns of other taxon leves;
ncbi_viral["Superkingdom"] = "viruses"
ncbi_viral["kingdom"] = None
ncbi_viral["order"] = None
ncbi_viral["phylum"] = None
ncbi_viral["class"] = None

# reorder the cols
ncbi_viral = ncbi_viral.iloc[:,[0,6,7,9,8,10,3,2,1,4,5]]
ncbi_viral.head()




,ident,Superkingdom,kingdom,phylum,order,class,Family,Genus,Species,Host,Isolation_Source
0,NC_079174.1,viruses,None,None,None,None,Chaseviridae,Loessnervirus,Erwinia phage Fifi44,Erwinia amylovora,NaN
1,NC_079175.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Escherichia phage PO103-1,Escherichia coli,NaN
2,NC_079176.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Escherichia phage MLP1,Escherichia coli CFT073,NaN
3,NC_079177.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Yersinia phage PYps3T,Yersinia pseudotuberculosis,NaN
4,NC_079178.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Yersinia phage PYps23T,Yersinia pseudotuberculosis,NaN


In [15]:
#sourmash needs the col names in lower case ):
ncbi_viral.rename(columns={"Superkingdom":"superkingdom",
                           "kingdom":"kingdom",
                           "Family":"family",
                           "Genus":"genus",
                           "Species":"species"}, inplace=True)
ncbi_viral.head()

,ident,superkingdom,kingdom,phylum,order,class,family,genus,species,Host,Isolation_Source
0,NC_079174.1,viruses,None,None,None,None,Chaseviridae,Loessnervirus,Erwinia phage Fifi44,Erwinia amylovora,NaN
1,NC_079175.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Escherichia phage PO103-1,Escherichia coli,NaN
2,NC_079176.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Escherichia phage MLP1,Escherichia coli CFT073,NaN
3,NC_079177.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Yersinia phage PYps3T,Yersinia pseudotuberculosis,NaN
4,NC_079178.1,viruses,None,None,None,None,Chaseviridae,Carltongylesvirus,Yersinia phage PYps23T,Yersinia pseudotuberculosis,NaN


In [16]:
#write the selected columns to a gz compressed file 
csv = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/tools/databases/ncbi-all-viruses-2023-08-lineages.csv.gz"
ncbi_viral.to_csv(csv, columns=ncbi_viral.columns)

In [17]:
#loading the file (written out above; if all went well)
csv = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/tools/databases/ncbi-all-viruses-2023-08-lineages.csv.gz"
ncbi_viral = pd.read_csv(csv)
ncbi_viral.head()

/tmp/ipykernel_3267704/674896567.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  ncbi_viral = pd.read_csv(csv)


,Unnamed: 0,ident,superkingdom,kingdom,phylum,order,class,family,genus,species,Host,Isolation_Source
0,0,NC_079174.1,viruses,NaN,NaN,NaN,NaN,Chaseviridae,Loessnervirus,Erwinia phage Fifi44,Erwinia amylovora,NaN
1,1,NC_079175.1,viruses,NaN,NaN,NaN,NaN,Chaseviridae,Carltongylesvirus,Escherichia phage PO103-1,Escherichia coli,NaN
2,2,NC_079176.1,viruses,NaN,NaN,NaN,NaN,Chaseviridae,Carltongylesvirus,Escherichia phage MLP1,Escherichia coli CFT073,NaN
3,3,NC_079177.1,viruses,NaN,NaN,NaN,NaN,Chaseviridae,Carltongylesvirus,Yersinia phage PYps3T,Yersinia pseudotuberculosis,NaN
4,4,NC_079178.1,viruses,NaN,NaN,NaN,NaN,Chaseviridae,Carltongylesvirus,Yersinia phage PYps23T,Yersinia pseudotuberculosis,NaN


### Get Taxid using ete3 package

In [3]:

ncbi = NCBITaxa()

In [6]:

def process_sourmash_kreports(kreportPath):
    """
    Process kreport files for multiple samples, extract relevant information, and consolidate it into a single DataFrame.

    Parameters:
    kreportPath (str): The directory containing kreport files for multiple samples.

    Returns:
    pd.DataFrame: A DataFrame containing consolidated kreport data.
    """

    kreport = pd.DataFrame()  # Initialize an empty DataFrame to store all samples' kreport data

    # Initialize the NCBI Taxonomy database
    ncbi = NCBITaxa()

    # Iterate through files in the kreportPath
    for samplefile in os.listdir(kreportPath):
        if samplefile.endswith("k31.gather.genbank.kreport.txt"):
            sample_kreport = pd.read_csv(os.path.join(kreportPath, samplefile), sep="\t", header=None)

            sample_kreport.rename(columns={
                0: "percent",
                1: "Domain_BPs",
                2: "species_BPs",
                3: "taxonomy",
                4: "taxid",
                5: "name"
            }, inplace=True)

            # Filter rows where 'taxonomy' is 'S' (indicating species)
            sample_kreport = sample_kreport.query("taxonomy == 'S'")

            taxid_list = []

            # Retrieve taxids for species names
            for index, name in enumerate(sample_kreport['name']):
                taxid_dict = ncbi.get_name_translator([name])
                for key, id in taxid_dict.items():
                    taxid_list.append(id[0])

            sample_kreport['taxid'] = taxid_list
            sample_kreport['sample'] = samplefile.split(".")[0]

            # Update the consolidated DataFrame with sample data
            kreport = pd.concat([kreport, sample_kreport])

    return kreport


In [7]:
kreportPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/results/sourmash/with-ncbi/scaled_300/output.sourmash-profiling/3-taxprofile"


In [16]:
kreport = process_sourmash_kreports(kreportPath)
#kreport.head()

/tmp/ipykernel_3813430/4052086245.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_kreport['taxid'] = taxid_list
/tmp/ipykernel_3813430/4052086245.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_kreport['sample'] = samplefile.split(".")[0]
/tmp/ipykernel_3813430/4052086245.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [17]:
kreport.head()

,percent,Domain_BPs,species_BPs,taxonomy,taxid,name,sample
47,0.52,76999,76999,S,186617,uncultured marine virus,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...
48,0.14,20000,20000,S,2202644,Microviridae sp.,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...
49,0.12,17000,17000,S,2202563,CRESS virus sp.,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...
50,0.07,10999,10999,S,1954248,Circoviridae sp.,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...
51,0.05,8000,8000,S,43761,Barfin flounder nervous necrosis virus,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...


#### Link sourmash viruses to the host-db

In [18]:

#load host db viruses
hostDbPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/scripts/virushostdb.tsv"
hostdb_viruses = pd.read_csv(hostDbPath, sep="\t", usecols=['virus tax id', "refseq id", "virus lineage", "host name", "DISEASE"])

#select only viruses with human host
hostdb_viruses.query(" `host name` == 'Homo sapiens'", inplace= True)
human_viruses_taxids = [x for x in hostdb_viruses['virus tax id'] ]

In [19]:
#
human_v= kreport.query("taxid.isin(@human_viruses_taxids)")
human_v.head()

,percent,Domain_BPs,species_BPs,taxonomy,taxid,name,sample
61,0.01,2000,2000,S,129951,Human mastadenovirus C,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...
77,0.01,1000,1000,S,11983,Norwalk virus,HCFKNDSX3_104762-001-002_71_TAAGCACA-TTAATGTC_...
63,0.03,1999,1999,S,129951,Human mastadenovirus C,HCFKNDSX3_104762-001-002_35_AGGTGCGA-GCCAGAAG_...
87,0.02,2000,2000,S,129951,Human mastadenovirus C,H7NKYDSX3_104762-001-003_06_CCAGTTAG-ATGACAGG_...
103,0.01,2000,2000,S,11983,Norwalk virus,HCFKNDSX3_104762-001-002_12_GAACCTAG-AATACCAG_...


In [20]:
kreport.shape

(848, 7)

In [21]:
sourmash_human_viruses = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/supplimentary_data/sourmash_k31_human_viruses.csv"
human_v.to_csv(sourmash_human_viruses, columns=human_v.columns)

sourmash_all_viruses = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/supplimentary_data/sourmash_k31_all_viruses.csv"
kreport.to_csv(sourmash_all_viruses, columns=kreport.columns)
